<a href="https://colab.research.google.com/github/kwakseoyeon/1test/blob/master/building_a_custom_chatbot_app_based_on_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai # openai 라이브러리를 설치합니다.
!pip install langchain

In [ ]:
import os

In [ ]:
from langchain.chat_models import ChatOpenAI

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

chat_model = ChatOpenAI()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

# "너는 코드 주석 전문가 처럼 행동해. 입력된 코드의 주석을 쉽고 상세하게 작성한다."
# Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            #"You are a nice chatbot having a conversation with a human."
            "너는 코드 주석 전문가 처럼 행동해. 입력된 코드의 주석을 쉽고 상세하게 작성한다."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key = "chat_history",
                                  return_messages = True)

conversation = LLMChain(
    llm = chat_model,
    prompt = prompt,
    memory = memory,
    verbose=True
)

def langchain_conversation_chain(input_text):

    output = conversation({"question": input_text})

    print(output)

    return output['text']

In [ ]:
def chat_with_user(user_message):
    ai_message = langchain_conversation_chain(user_message)
    return ai_message

while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message)
    print(f" A I > {ai_message}")

USER > for i in range(1, 10): \n print(i)


> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 코드 주석 전문가 처럼 행동해. 입력된 코드의 주석을 쉽고 상세하게 작성한다.
Human: for i in range(1, 10): \n print(i)

> Finished chain.
{'question': 'for i in range(1, 10): \\n print(i)', 'chat_history': [HumanMessage(content='for i in range(1, 10): \\n print(i)'), AIMessage(content='```python\n# 1부터 9까지의 숫자를 반복적으로 출력하는 반복문\nfor i in range(1, 10):\n    print(i)  # 현재 순회 중인 숫자를 출력\n```')], 'text': '```python\n# 1부터 9까지의 숫자를 반복적으로 출력하는 반복문\nfor i in range(1, 10):\n    print(i)  # 현재 순회 중인 숫자를 출력\n```'}
 A I > ```python
# 1부터 9까지의 숫자를 반복적으로 출력하는 반복문
for i in range(1, 10):
    print(i)  # 현재 순회 중인 숫자를 출력
```
USER > quit
